<b>Segmenting and Clustering Neighborhoods in Toronto - I have used one notebook for all three questions</b>

Part 1 of 3 - Create Pandas Dataframe

In [145]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library
!pip install beautifulsoup4
from bs4 import BeautifulSoup
!pip install lxml
import lxml
print('Libraries imported.')

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Libraries imported.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [31]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(website_url,'lxml')

In [43]:
table = soup.find("table")
print(table.text)



Postcode
Borough
Neighborhood


M1A
Not assigned
Not assigned


M2A
Not assigned
Not assigned


M3A
North York
Parkwoods


M4A
North York
Victoria Village


M5A
Downtown Toronto
Harbourfront


M6A
North York
Lawrence Heights


M6A
North York
Lawrence Manor


M7A
Downtown Toronto
Queen's Park


M8A
Not assigned
Not assigned


M9A
Queen's Park
Not assigned


M1B
Scarborough
Rouge


M1B
Scarborough
Malvern


M2B
Not assigned
Not assigned


M3B
North York
Don Mills North


M4B
East York
Woodbine Gardens


M4B
East York
Parkview Hill


M5B
Downtown Toronto
Ryerson


M5B
Downtown Toronto
Garden District


M6B
North York
Glencairn


M7B
Not assigned
Not assigned


M8B
Not assigned
Not assigned


M9B
Etobicoke
Cloverdale


M9B
Etobicoke
Islington


M9B
Etobicoke
Martin Grove


M9B
Etobicoke
Princess Gardens


M9B
Etobicoke
West Deane Park


M1C
Scarborough
Highland Creek


M1C
Scarborough
Rouge Hill


M1C
Scarborough
Port Union


M2C
Not assigned
Not assigned


M3C
North York
Flemingdon Park

In [54]:
output_rows = []

""


In [59]:
for table_row in table.find_all('tr'):
    columns = table_row.find_all('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)
output_rows

[[],
 ['M1A', 'Not assigned', 'Not assigned\n'],
 ['M2A', 'Not assigned', 'Not assigned\n'],
 ['M3A', 'North York', 'Parkwoods\n'],
 ['M4A', 'North York', 'Victoria Village\n'],
 ['M5A', 'Downtown Toronto', 'Harbourfront\n'],
 ['M6A', 'North York', 'Lawrence Heights\n'],
 ['M6A', 'North York', 'Lawrence Manor\n'],
 ['M7A', 'Downtown Toronto', "Queen's Park\n"],
 ['M8A', 'Not assigned', 'Not assigned\n'],
 ['M9A', "Queen's Park", 'Not assigned\n'],
 ['M1B', 'Scarborough', 'Rouge\n'],
 ['M1B', 'Scarborough', 'Malvern\n'],
 ['M2B', 'Not assigned', 'Not assigned\n'],
 ['M3B', 'North York', 'Don Mills North\n'],
 ['M4B', 'East York', 'Woodbine Gardens\n'],
 ['M4B', 'East York', 'Parkview Hill\n'],
 ['M5B', 'Downtown Toronto', 'Ryerson\n'],
 ['M5B', 'Downtown Toronto', 'Garden District\n'],
 ['M6B', 'North York', 'Glencairn\n'],
 ['M7B', 'Not assigned', 'Not assigned\n'],
 ['M8B', 'Not assigned', 'Not assigned\n'],
 ['M9B', 'Etobicoke', 'Cloverdale\n'],
 ['M9B', 'Etobicoke', 'Islington\n'],


In [130]:
postalcodes = pd.DataFrame(output_rows, columns = ['PostalCode', 'Borough', 'Neighborhood'])
postalcodes['Neighborhood'] = postalcodes['Neighborhood'].str.replace('\n', '')
postalcodes.drop(0, axis=0, inplace = True)
postalcodes = postalcodes[postalcodes.Borough != 'Not assigned'].reset_index(drop = True)

In [131]:
postalcodes.drop_duplicates(subset = "Neighborhood", keep = 'first', inplace = True) 

In [133]:
mask = postalcodes['Neighborhood'] == 'Not assigned'
postalcodes['Neighborhood'][mask] = postalcodes['Borough'][mask]

In [271]:
postalcodes = postalcodes.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list)
postalcodes = postalcodes.sample(frac = 1).reset_index()
postalcodes['Neighborhood'] = postalcodes['Neighborhood'].str.join(',')
postalcodes.shape

(103, 3)

<b>Part 2 of 3 - Create Dataframe with CSV file. I was unable to use the geocoder after many failed attempts to get results.</b>

In [162]:
coords = pd.read_csv("https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")
ps_coords = postalcodes
ps_coords = ps_coords.join(coords.set_index('Postal Code'), on='PostalCode')

In [238]:
ps_coords.head()
print(ps_coords.shape)

(103, 5)


In [161]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


In [165]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ps_coords['Latitude'], ps_coords['Longitude'], ps_coords['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<b>Part 3 of 3 - Segment and Cluster the neighborhoods using kMeans</b>

In [166]:
CLIENT_ID = '2PDWWJRDIEK1WIIAOW5D44NERINP5G3TCEZCJXVKOO3VVNYM' # your Foursquare ID
CLIENT_SECRET = 'NXHOS5LMMVQKNVLAQ3I5RI4CANTIKLREML50DXIDPTXEKNA3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [248]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [251]:
LIMIT = 100
toronto_venues = getNearbyVenues(names = ps_coords['Neighborhood'],
                                 latitudes = ps_coords['Latitude'], longitudes = ps_coords['Longitude'])

L'Amoreaux West
Humewood-Cedarvale
Bathurst Manor,Downsview North,Wilson Heights
Glencairn
Kingsview Village,Martin Grove Gardens,Richview Gardens,St. Phillips
Scarborough Village
Ryerson,Garden District
The Beaches West,India Bazaar
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Clairlea,Golden Mile,Oakridge
Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown
Emery,Humberlea
Agincourt North,L'Amoreaux East,Milliken,Steeles East
Weston
Fairview,Henry Farm,Oriole
Humber Bay Shores,Mimico South,New Toronto
Victoria Village
Clarks Corners,Sullivan,Tam O'Shanter
Newtonbrook,Willowdale
Caledonia-Fairbanks
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Swansea
Woodbine Gardens,Parkview Hill
Alderwood,Long Branch
Maryvale,Wexford
Cedarbrae
Woodbine Heights
CFB Toronto,Downsview East
Forest Hill North,Forest Hill West
Studio District
Northwood Park,York University
The Beaches
Roselawn
Lawrence Park
Birch Cliff,C

In [252]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop('Neighborhood', axis=1, inplace = True)
# add neighborhood column back to dataframe
toronto_onehot.insert(0, "Neighborhood", toronto_venues['Neighborhood'], True) 

In [253]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(100, 274)

In [199]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [257]:
num_top_venues = 10
indicators = ['st','nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,Burger Joint,Thai Restaurant,Restaurant,Sushi Restaurant,Asian Restaurant,Bakery
1,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Clothing Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Coffee Shop,Arts & Crafts Store,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Video Store,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Sandwich Place,Beer Store,Pharmacy,Airport Service,Airport
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Skating Rink,Dance Studio,Pharmacy,Pool,Pub,Sandwich Place,Gym,American Restaurant


In [258]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0])

In [259]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = ps_coords

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,0.0,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Gym Pool,Grocery Store,Pizza Place,Coffee Shop,Breakfast Spot,Supermarket,Sandwich Place
1,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,0.0,Hockey Arena,Trail,Field,Playground,Tennis Court,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,M3H,North York,"Bathurst Manor,Downsview North,Wilson Heights",43.754328,-79.442259,0.0,Coffee Shop,Pet Store,Sandwich Place,Sushi Restaurant,Middle Eastern Restaurant,Bank,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner
3,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Japanese Restaurant,Playground,Pub,Bakery,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
4,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724,2.0,Mobile Phone Shop,Pizza Place,Park,Bus Line,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Playground,Smoke Shop,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
6,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bakery,Theater,Pizza Place,Burger Joint,Fast Food Restaurant
7,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0.0,Sandwich Place,Park,Food & Drink Shop,Pub,Burger Joint,Burrito Place,Board Shop,Fast Food Restaurant,Italian Restaurant,Intersection
8,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",43.643515,-79.577201,0.0,Pharmacy,Coffee Shop,Beer Store,Shopping Plaza,Café,Liquor Store,Pizza Place,Diner,Event Space,Ethiopian Restaurant
9,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,0.0,Bakery,Bus Line,Fast Food Restaurant,Metro Station,Soccer Field,Intersection,Park,German Restaurant,Dessert Shop,Empanada Restaurant


In [261]:
toronto_merged.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int')

In [262]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<b>I used kmeans to segment and cluster the neighborhoods the same as in the NY assignment. I'm not sure this the best method for segmenting by venue proximity, but you do see some patterns</b></i>

Cluster 0 has a good mix of different venues nearby, probably in a <b><i>Dense Residential or Urban</i></b> area

In [267]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,L'Amoreaux West,0,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Gym Pool,Grocery Store,Pizza Place,Coffee Shop,Breakfast Spot,Supermarket,Sandwich Place
1,Humewood-Cedarvale,0,Hockey Arena,Trail,Field,Playground,Tennis Court,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Bathurst Manor,Downsview North,Wilson Heights",0,Coffee Shop,Pet Store,Sandwich Place,Sushi Restaurant,Middle Eastern Restaurant,Bank,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner
3,Glencairn,0,Japanese Restaurant,Playground,Pub,Bakery,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
5,Scarborough Village,0,Playground,Smoke Shop,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
6,"Ryerson,Garden District",0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bakery,Theater,Pizza Place,Burger Joint,Fast Food Restaurant
7,"The Beaches West,India Bazaar",0,Sandwich Place,Park,Food & Drink Shop,Pub,Burger Joint,Burrito Place,Board Shop,Fast Food Restaurant,Italian Restaurant,Intersection
8,"Bloordale Gardens,Eringate,Markland Wood,Old B...",0,Pharmacy,Coffee Shop,Beer Store,Shopping Plaza,Café,Liquor Store,Pizza Place,Diner,Event Space,Ethiopian Restaurant
9,"Clairlea,Golden Mile,Oakridge",0,Bakery,Bus Line,Fast Food Restaurant,Metro Station,Soccer Field,Intersection,Park,German Restaurant,Dessert Shop,Empanada Restaurant
10,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0,Grocery Store,Video Store,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Sandwich Place,Beer Store,Pharmacy,Airport Service,Airport


Cluster 1 has only one postal code neighborhood, possibly a <b><i>Class of Its Own</i></b>.

In [268]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,"Silver Hills,York Mills",1,Cafeteria,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Stadium


Cluster 2 neighborhoods are all very close to parks, and may be <b><i>Residential, or Close to Nature</i></b>

In [269]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Kingsview Village,Martin Grove Gardens,Richvie...",2,Mobile Phone Shop,Pizza Place,Park,Bus Line,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
12,"Agincourt North,L'Amoreaux East,Milliken,Steel...",2,Park,Playground,Coffee Shop,Arts & Crafts Store,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant
13,Weston,2,Park,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
19,Caledonia-Fairbanks,2,Park,Women's Store,Fast Food Restaurant,Market,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
27,"CFB Toronto,Downsview East",2,Airport,Park,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
28,"Forest Hill North,Forest Hill West",2,Trail,Park,Jewelry Store,Sushi Restaurant,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
33,Lawrence Park,2,Park,Bus Line,Swim School,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
35,Parkwoods,2,Food & Drink Shop,Park,Construction & Landscaping,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
49,"The Kingsway,Montgomery Road,Old Mill North",2,Park,River,Smoke Shop,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
57,East Toronto,2,Park,Coffee Shop,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


Class 3 neighborhoods are both identical... which seems counter-intuitive, but may have had the same <b><i>City Planning Concept or Baseball Field venues</b></i> when created.

In [270]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"Emery,Humberlea",3,Baseball Field,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Festival
75,"Humber Bay,King's Mill Park,Kingsway Park Sout...",3,Baseball Field,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Festival
